In [1]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
# python func in tf func umwandeln
@tf.function
def cube(x):
    return x ** 3

# un umgekehrt
tf.cube.python_function(2)

In [13]:
# in tensorflow data api dreht sich alles um dataset
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)

In [9]:
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [14]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


wichtig: die dataset methoden modifizieren nicht sie erstellen neue datasets

In [11]:
dataset = dataset.repeat(3)
for item in dataset:
    print(item) #

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype

In [12]:
dataset = dataset.repeat(3).batch(7) #drop_remainder=True wenn alle batches gleich bleiben sollen
for item in dataset:
    print(item) #

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9 0 1 2 3 4], shape=(7,), dtype=int32)
tf.Tensor([5 6 7 8 9 0 1], shape=(7,), dtype=int32)
tf.Tensor([2 3 4 5 6 7 8], shape=(7,), dtype=int32)
tf.Tensor([9 0 1 2 3 4 5], shape=(7,), dtype=int32)
tf.Tensor([6 7 8 9 0 1 2], shape=(7,), dtype=int32)
tf.Tensor([3 4 5 6 7 8 9], shape=(7,), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)


In [15]:
dataset = dataset.map(lambda x: x * 2) # addiert jedes elemnt mit 2

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [19]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)


In [ ]:
dataset = dataset.apply(tf.data.experimental.unbatch()) # apply wendet auch das ganze dataset an
dataset = dataset.filter(lambda x: x < 10) 

In [20]:
for item in dataset.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


In [22]:
#mischen der daten
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


quelldaten durchmischen

gegeben: train, valid and testdataset

jedes davon ist nochmal in dateien unterteilt

In [ ]:
train_filepaths = "path aller dateien zu train"

In [ ]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42) #gibt dataset mit durchmischten filenames zurück
n_readers = 5

dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1), # skip header von zb.
    cycle_length=n_readers # nimmt sich immer 5 zufällige dateien und fügt sie zusammen 
) # dataset enthält dann den durschmischen test datensatz in bytestrings welches noch sklaliert und geparse werden muss

X_mean, X_std = []

n_inputs = 8 #features

def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)] #standardwerte für jede zeile, für features null und target leeres array (tf wirft exeption wenn leer)
    fields = tf.io.decode_csv(line, record_defaults=defs) # gibt liste mit skalar tensoren zurück
    x = tf.stack(fields[:-1]) # müssen deshalb wieder in einem 1 dim array zusammengefasst werden
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

In [ ]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5, n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = filepath_dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1), # skip header von zb.
        cycle_length=n_readers, # nimmt sich immer 5 zufällige dateien und fügt sie zusammen
        num_parallel_calls=n_read_threads
    )
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1) #aus performance gründen wir preprocess daten während daten trainiert werden

In [ ]:
# so kann dann das training weiter aussehen

train_set = csv_reader_dataset(train_filepaths)
valid_set = ""
test_set = ""

model = keras.models.Sequential()
model.compile()
model.fit(train_set, epoch=10, validation_data=valid_set)

model.evaluate(test_set)
new_set = test_set.take(3).map(lambda X, y: X)
model.predict(new_set)

große datensätze werden bevorzugt in tfrecord geschrieben

In [23]:
options = tf.io.TFRecordOptions(compression_type="GZIP") # kann zusätzlich in den writer gegeben werden 
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"erster dataset")
    f.write(b"zweiter dataset")

In [24]:
filepath = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepath) # zusätzlich kann beim lesen den compression_type übergeben werden
for item in dataset:
    print(item)

tf.Tensor(b'erster dataset', shape=(), dtype=string)
tf.Tensor(b'zweiter dataset', shape=(), dtype=string)
